In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/0v/2jlpljqd1msbtz5s_xyw4yj00000gn/T/ipykernel_11857/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/guyparsadanov/Downloads/Iron-Hack-Work/W4/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
from sklearn.model_selection import train_test_split

# Assuming the dataset has two columns: text data in the first column and labels in the second
X = data.iloc[:, 0]     # Text data (features)
y = data.iloc[:, 1]     # Labels (e.g., 'spam' or 'ham')

# Split the data into 80% training and 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## Data Preprocessing

In [5]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
import re

def clean_html(text):
    # Step 1: Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)  # Remove <script> tags
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)    # Remove <style> tags

    # Step 2: Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)               # Remove comments

    # Step 3: Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)                                   # Remove all other HTML tags

    # Optional: Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply the clean_html function to each element of X_train
cleanX = X_train.apply(clean_html)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [8]:
import re

def preprocess_text(text):
    # Remove special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove single characters
    text = re.sub(r'\b\w\b', '', text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()
    return text

# Apply the preprocessing function to cleanX
cleanX_preprocessed = cleanX.apply(preprocess_text)

In [9]:
def remove_prefixed_b(text):
    # Remove any prefixed 'b' from the start of the string
    if text.startswith('b'):
        text = text[1:]
    return text

# Apply the function to the preprocessed data
cleanX_final = cleanX_preprocessed.apply(remove_prefixed_b)

## Now let's work on removing stopwords
Remove the stopwords.

In [10]:
from nltk.corpus import stopwords

# Download stopwords if not already available
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the stopword removal function
cleanX_no_stopwords = cleanX_final.apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/guyparsadanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [11]:
from nltk.stem import WordNetLemmatizer

# Download WordNet data if not already available
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Apply the lemmatization function
cleanX_lemmatized = cleanX_no_stopwords.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/guyparsadanov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer(max_features=10)  # Limit to top 10 words for simplicity

# Fit and transform the lemmatized data
X_bow = vectorizer.fit_transform(cleanX_lemmatized)

# Convert to a DataFrame for better visualization
X_bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Display the resulting DataFrame
X_bow_df.head()

,account,bank,business,company,country,fund,million,money,transaction,transfer
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,1,5,0,0,0,3,2,1,1,1


In [14]:
from sklearn.model_selection import train_test_split

# Assuming cleanX_lemmatized is the preprocessed text and y contains the labels
data = pd.DataFrame({'preprocessed_text': cleanX_lemmatized, 'label': y})

# Split into training (80%) and validation (20%) sets
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

In [18]:
# Replace non-string values with an empty string
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(lambda x: x if isinstance(x, str) else "")

# Verify that all values are now strings
assert data_val['preprocessed_text'].apply(lambda x: isinstance(x, str)).all()

## Extra features

In [19]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,preprocessed_text,label,money_mark,suspicious_words,text_len
29,regard mr nelson smithkindly reply private ema...,1,1,0,75
535,able reach oscar supposed send pdb receive,0,1,0,42
695,huma abedin bim checking pat work jack jake re...,0,1,0,79
557,announced monday cant today,0,1,0,27
836,ank africaagence san pedro bp san pedro cote d...,1,1,1,1066


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10)  # Limit to top 10 features

# Fit and transform the lemmatized data
X_tfidf = tfidf_vectorizer.fit_transform(cleanX_lemmatized)

# Convert to a DataFrame for better visualization
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the resulting DataFrame
X_tfidf_df.head()

,account,bank,business,company,country,fund,million,money,transaction,transfer
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.152539,0.783971,0.0,0.0,0.0,0.439495,0.304447,0.141508,0.160989,0.174171


## And the Train a Classifier?

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Initialize the Multinomial Naive Bayes classifier
classifier = MultinomialNB()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train the classifier
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

ValueError: Found input variables with inconsistent numbers of samples: [800, 1000]

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code